## 기본 통계

### 재료(ingredient)-식품(base_food) 매핑

- 총 435종 부여 (coverage: 전체 base_foods 아이템 중 `10.0%`)
- 매핑되지 않은 ingredient rows: 3,105 / 6,761 (`46.0%`)
- 전체 식품 출현 빈도는 Poisson 분포를 따를 것으로 보임 


### 관련 R Code: ingredients, base_foods CSV 파일 가공 

In [1]:
## read ingredients, base_foods

ingredients <- read.csv('/Users/apollos/Desktop/git/work-vh/ingredients/ingredients.csv')
base_foods <- read.csv('/Users/apollos/Desktop/git/work-vh/ingredients/basefoods_trimmed.csv')

ingredients$bf_code[is.na(ingredients$bf_code)] <- 'UNDEFINED'
ingredients$bf_code <- as.factor(ingredients$bf_code)

bf.code.desc <- base_foods$name
bf.code.label <- base_foods$label
names(bf.code.desc) <- as.character(base_foods$code)
names(bf.code.label) <- as.character(base_foods$code)

# v <- as.vector(by(ingredients, ingredients$bf_code, nrow))
# names(v) <- levels(ingredients$bf_code)
# v <- sort(v, decreasing = T)
# ingredients$bf_code <- factor(ingredients$bf_code, levels=names(v))

## 고빈도 식품(base_food) 아이템

![고빈도 식품 30종](highest_bf_bar.png)

### 관련 R Code: transform, dump

In [2]:
## Check ingredients mapped to base_food

ing_actual <- ingredients[ingredients$bf_code != 'UNDEFINED',]
ing_actual$bf_code <- as.factor(ing_actual$bf_code)
ing.bf.freq <- as.vector(by(ing_actual, ing_actual$bf_code, nrow))
names(ing.bf.freq) <- levels(ing_actual$bf_code)
ing.bf.freq<- sort(ing.bf.freq, decreasing = T)
bf.code.ordered <- names(ing.bf.freq)

names(ing.bf.freq) <- bf.code.desc[bf.code.ordered]
ing_actual$bf_desc <- bf.code.desc[as.character(ing_actual$bf_code)]
ing_actual$bf_desc <- factor(ing_actual$bf_desc, levels=names(ing.bf.freq))
ing_actual$bf_code <- factor(ing_actual$bf_code, levels=bf.code.ordered)

ing.frame <- data.frame(code=bf.code.ordered, freq=ing.bf.freq, desc=names(ing.bf.freq), label=bf.code.label[bf.code.ordered])
write.csv(ing.frame, '/Users/apollos/Desktop/git/work-vh/ingredients/ingredients_mapped.csv', row.names=F)

### 분석 결과 (상위 15건)

In [3]:
head(ing.frame, 15)
#ing.bf.freq[1:15]
#bf.code.label[bf.code.ordered[1:15]]

,code,freq,desc,label
"양파, 생것, 국내산",61108000,172,"양파, 생것, 국내산",양파
"마늘, 구근, 생것(국내산)",61052010,154,"마늘, 구근, 생것(국내산)",마늘
"설탕, 백설탕",31005020,139,"설탕, 백설탕",백설탕
"소금, 식염",161016010,131,"소금, 식염",소금
"간장, 진간장",161001010,125,"간장, 진간장",진간장
"후추, 분말",161041000,110,"후추, 분말",후추
"계란, 전란, 생것",101001000,106,"계란, 전란, 생것",계란
"파, 대파",61151010,101,"파, 대파",대파
고추가루,161004010,88,고추가루,고추가루
참기름,142012000,68,참기름,참기름


## 매핑 실패 사례

### 매핑 실패 사례 추출 및 데이터 전처리

In [4]:
## failed in mapping (undefined)

ing_undefined <- ingredients[ingredients$bf_code == 'UNDEFINED',]
ing_undefined$name <- as.factor(ing_undefined$name)
ing.undefined.freq <- as.vector(by(ing_undefined, ing_undefined$name, nrow))
names(ing.undefined.freq) <- levels(ing_undefined$name)
ing.undefined.freq <- sort(ing.undefined.freq, decreasing = T)

### 상위 사례 관찰
- base_food에 존재하지 않는 경우 (`물`)
- Pretty Print를 위한 재료 카테고리 구분 태그 역할 (`<양념>`)
- 많은 사례의 경우, 매핑될 것으로 보이는데 매핑되지 않음 => 이유는?

In [5]:
ing.undefined.freq[1:15]

물     소금   <양념>     후추     양파 다진마늘   참기름     통깨 
     126       76       72       66       58       53       50       47 
청양고추     설탕 올리고당     간장     당근     대파     계란 
      45       42       36       34       33       33       30

### 동일 재료명에 대한 매핑 실패(?)

- 실제 실패는 아니며, 레시피 작성 UI에서 사용자가 시스템에서 제시한 base_food를 선택해 줘야 실제 등록 
- 선택하지 않은 경우 매핑되지 않음 (그리고 점차 선택하지 않는 경우가 늘어나고 있음!)

### 매핑 관련 이슈

#### 정제/특화된 재료 컨셉 필요

`base_food`로 접근했으나, 이러한 conceptualization(abstraction)을 위해서는 더 정밀한 태그가 붙어야 함

- 관점별 카테고리 추가

- 속성 추가
> 예) 재료에는 제한된 유형의 요리 방법이 있을 것

- 재료간 연관관계는?
> 고기 요리에는 주로 쓰이는 양념류와, 고기 냄새를 제거하기 위한 재료등이 등장할 것. 이처럼 함께 나타나는 연관관계 + 재료 유형

#### 정제 필요성

매칭을 위해 검색 기준이 되는 텍스트 정제 등 필요
- 띄어쓰기 통일 (검색시 없애는 방향으로)
- `nb_nickname` 정제 필요 
- `()`로 둘러쌓인 텍스트는 이름에서 빼고 검색해줘야 할 것
- 재료 뒤 각종 설명을 위해 늘여쓴 내용도 필터링해야 함 (언어분석 필요)
- `<>`로 둘러쌓인 태그는 분석에서 filter-out해야 함

### 가능성
- 현재 이를 위해 DB query를 (ruby code로) 돌리고 있는데, 이미 구해진 고빈도 매핑은 바로 기계적으로 정해도 문제 없을 것 같음
- <양념>, <소스> 등의 태그를 재료의 (추가적) 메타데이터를 구하는 데 활용할 수 있음



### 관련 데이터 탐색 (참고)

In [6]:
nrow(ingredients[ingredients$name=='고추장' & ingredients$bf_code=='UNDEFINED',]) / nrow(ingredients[ingredients$name=='고추장',])
ingredients[ingredients$name=='고추장', c('recipe_id', 'bf_code', 'amount')][30:40,]

[1] 0.3536585

,recipe_id,bf_code,amount
2579,327,UNDEFINED,.
2585,328,UNDEFINED,.
2771,348,UNDEFINED,1 숟가락
2915,361,UNDEFINED,2 숟가락
2943,369,UNDEFINED,2 큰술
3046,383,UNDEFINED,1 큰술
3064,384,161005000,1 큰술
3174,398,161005000,6 숟가락
3475,432,161005000,1 Ts
3524,438,UNDEFINED,1작은술


In [7]:
# find freq. of ingredients only for categorized view
ing.undefined.freq[substr(names(ing.undefined.freq), 1,1) == '<'][1:10]

<양념>            <소스>          <양념장>            <밑간> 
               72                13                12                 3 
       <고기밑간> <비빔국수 양념장>            <육수>            <장식> 
                2                 2                 2                 2 
          < 양념>        <강정시럽> 
                1                 1

> 개념적으로 재료 유형/카테고리에 대한 메타데이터가 필요했으나, 현재 스키마가 제공하지 못해 발생한 편법

### Graph DB (Neo4J) 

#### 연관관계 기반 재료명 매핑 테스트중

#### Eg: `설탕`에 대한 base_food (개념) 찾기 

##### 내부 QL: Cypher 코드 
```
MATCH (b:Bf), (a1:BfAlias)
WHERE a1.name =~ '*설탕' 
    AND (NOT a1.name IN split(b.name, ', ') OR a1.name=b.label) 
    AND (b)-[:AKA]->(a1)
RETURN distinct b
```
##### 실행 결과
![Cypher 실행 결과](graph-cypher-result.png)

## 재료별 매핑

### 일반적 이슈

- UNDEFINED가 다수 발견
    - base_foods내 존재하지 않는 경우, 사용자 미지정한 경우
- 동일한 재료명의 매핑 여부가 갈릴 때 많음
    - 고빈도 재료명의 경우 이때 이미 동일 재료명에서 찾은 매핑을 그대로 일괄 적용해도 좋겠다고 생각함


### 고빈도  재료명 사례: 계란

주로 base_food `101001000`에 매핑되나, 다양한 변형 사례 발견
- 띄어쓰기 관련 통일해서 (즉 normalization 기준을 줘서) nickname을 관리해야 함

### 고빈도 재료명 사례: 돼지고기

다양한 재료 유형 발견됨 (돼지고기 관련 재료 코드는 `920*`임)

이름 매칭 이슈 
- 괄호에 둘러쌓인 텍스트는 trimming해서 매칭하는 게 좋겠음

돼지고기 관련 basefood의 이름 매핑 사례는 아래 그래프와 같음
![매핑 사례](graph-pork-aliases.png)



### 관련 탐색용 R 코드

In [8]:
# view ingredient-basefood mappings
ingredients$bf_mapping <- paste(ingredients$name, '-', ingredients$bf_code, sep='')
ingredients$bf_mapping <- factor(ingredients$bf_mapping, levels=sort(unique(ingredients$bf_mapping)))

mapping.count <- as.vector(by(ingredients, ingredients$bf_mapping, nrow))
names(mapping.count) <- levels(ingredients$bf_mapping)

In [9]:
sort(mapping.count[substr(names(mapping.count),1,1) != '<'], decreasing = T)[1:15]
mapping.count[substr(names(mapping.count),1,2) == '계란']
mapping.count[substr(names(mapping.count),1,4) == '돼지고기']

양파-61108000     소금-161016010       물-UNDEFINED      마늘-61052010 
               169                130                126                108 
    후추-161041000     계란-101001000    백설탕-31005020      대파-61151010 
               104                 96                 92                 81 
    소금-UNDEFINED   진간장-161001010 고추가루-161004010   참기름-142012000 
                76                 74                 70                 68 
    후추-UNDEFINED     양파-UNDEFINED   고추장-161005000 
                66                 58                 53

계란 노른자-101003000       계란 삶은것-UNDEFINED 
                          1                           1 
             계란-101001000              계란-UNDEFINED 
                         96                          30 
계란노른자 삶은것-101003010        계란노른자-101003000 
                          1                          11 
       계란노른자-UNDEFINED        계란후라이-101009000 
                          1                           1 
         계란흰자-101002000 
                          3

돼지고기 등갈비살-UNDEFINED                 돼지고기 등심-92003000 
                                     1                                      3 
               돼지고기 목살-UNDEFINED  돼지고기 삼겹살 (혹은 목살)-UNDEFINED 
                                     1                                      1 
                돼지고기 안심-92006000                      돼지고기-92021000 
                                     1                                      3 
                    돼지고기-UNDEFINED   돼지고기(목살 혹은 삼겹살)-UNDEFINED 
                                     6                                      1 
            돼지고기(잡채용)-UNDEFINED                  돼지고기간것-92006030 
                                     1                                      2 
              돼지고기다진것-UNDEFINED 돼지고기등심(잡채용 돼지고기)-92003000 
                                     1                                      1 
                 돼지고기목살-92021000 
                                     1